In [1]:
!uv add toyaikit

Resolved 177 packages in 3.35s
Installed 2 packages in 136ms
 + genai-prices==0.0.52
 + toyaikit==0.0.9


In [1]:
from gitsource import GithubRepositoryDataReader, chunk_documents
from minsearch import AppendableIndex


reader = GithubRepositoryDataReader(
    repo_owner="evidentlyai",
    repo_name="docs",
    allowed_extensions={"md", "mdx"},
)
files = reader.read()

parsed_docs = [doc.parse() for doc in files]
chunked_docs = chunk_documents(parsed_docs, size=3000, step=1500)

index = AppendableIndex(
    text_fields=["title", "description", "content"],
    keyword_fields=["filename"]
)
index.fit(chunked_docs)

In [2]:
def search(query):
    results = index.search(
        query=query,
        num_results=5
    )
    return results

search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the documentation database for relevant results based on a query string.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The search query to look up in the index"
            }
        },
        "required": [
            "query"
        ]
    }
}

In [3]:
def add_entry(filename, title, description, content):
    entry = {
        'start': 0,
        'content': content,
        'title': title,
        'description': description,
        'filename': filename,
    }
    index.append(entry)
    return "OK"

add_entry_tool = {
    "type": "function",
    "name": "add_entry",
    "description": "Add a new documentation entry to the index.",
    "parameters": {
        "type": "object",
        "properties": {
            "filename": {
                "type": "string",
                "description": "The source filename associated with the entry"
            },
            "title": {
                "type": "string",
                "description": "The title of the documentation entry"
            },
            "description": {
                "type": "string",
                "description": "A short description summarizing the entry"
            },
            "content": {
                "type": "string",
                "description": "The full content of the documentation entry"
            }
        },
        "required": [
            "filename",
            "title",
            "description",
            "content"
        ]
    }
}


In [4]:
instructions = """
You're a documentation assistant. 

Answer the user question using the documentation knowledge base

Make 3 iterations:

1) in the first iteration, perform one search
2) in the second interation, analyze the results from the previous search
   and perform 2 more searches
3) synthesise the results into the output

IMPORTANT: at each step, give an explanation of why you want to perform 
search for this particular search query. It should be 2-3 sentences explaining
the logic of your decision.

Use only facts from the knowledge base when answering.
If you cannot find the answer, inform the user.

Our knowledge base is entirely about Evidently, so you don't need to 
include the word 'evidently' in search results
"""

In [5]:
from toyaikit.tools import Tools

In [6]:
agent_tools = Tools()
agent_tools.add_tool(search, search_tool)
agent_tools.add_tool(add_entry, add_entry_tool)

In [7]:
agent_tools.get_tools()

[{'type': 'function',
  'name': 'search',
  'description': 'Search the documentation database for relevant results based on a query string.',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'The search query to look up in the index'}},
   'required': ['query']}},
 {'type': 'function',
  'name': 'add_entry',
  'description': 'Add a new documentation entry to the index.',
  'parameters': {'type': 'object',
   'properties': {'filename': {'type': 'string',
     'description': 'The source filename associated with the entry'},
    'title': {'type': 'string',
     'description': 'The title of the documentation entry'},
    'description': {'type': 'string',
     'description': 'A short description summarizing the entry'},
    'content': {'type': 'string',
     'description': 'The full content of the documentation entry'}},
   'required': ['filename', 'title', 'description', 'content']}}]

In [8]:
from toyaikit.chat.runners import OpenAIResponsesRunner
from toyaikit.llm import OpenAIClient
from openai import OpenAI

llm_client = OpenAIClient(
    model="gpt-4o-mini",
    client=OpenAI()
)

In [9]:
agent = OpenAIResponsesRunner(
    tools=agent_tools,
    developer_prompt=instructions,
    # chat_interface=,
    llm_client=llm_client
)

In [10]:
result = agent.loop('how can I create evidently dahsbord?')

In [16]:
print(result.last_message)

Here’s a comprehensive guide on how to create and manage a dashboard, integrating insights from the results of my previous searches. Understanding both how to add panels and manage them will give you a robust view of your dashboard functionalities.

### Creating a Dashboard

1. **Initial Setup**:
   - Ensure you are connected to your project, either in Evidently Cloud or on another platform where you have access.

2. **Adding Tabs**:
   - In the dashboard interface, enter "Edit" mode.
   - To add a new tab, click the "+" icon. You can create a custom tab or use a pre-built template that suits your needs.

   ```python
   project.dashboard.add_tab("My New Tab")
   ```

3. **Adding Panels**:
   - Enter "Edit" mode again and click on the "Add Panel" button.
   - You can add various types of panels (text, counters, pie charts, and various plots) to visualize your metrics.

   Example of adding a simple text panel:
   ```python
   project.dashboard.add_panel(
       DashboardPanelPlot(
    

In [15]:
result.cost.total_cost

Decimal('0.00308835')

In [17]:
from toyaikit.chat.interface import IPythonChatInterface
from toyaikit.chat.runners import DisplayingRunnerCallback

In [19]:
chat_interface = IPythonChatInterface()
runner_callback = DisplayingRunnerCallback(chat_interface=chat_interface) 

In [20]:
result = agent.loop(
    'how can I create evidently dahsbord?',
    callback=runner_callback
)

-> Response received


-> Response received


-> Response received


In [21]:
result2 = agent.loop(
    'show me the code',
    callback=runner_callback,
    previous_messages=result.all_messages
)

-> Response received


In [24]:
agent = OpenAIResponsesRunner(
    tools=agent_tools,
    developer_prompt=instructions,
    chat_interface=chat_interface,
    llm_client=llm_client
)

In [25]:
result = agent.run()

You: dahsbord pls


-> Response received


-> Response received


-> Response received


You: show me more cod


-> Response received


You: stop


Chat ended.


In [27]:
result.cost.total_cost

Decimal('0.00544875')